# Przetwarzanie warunkowe

Umożliwia nakładanie warunków na wykonywanie się kodu, które działają wyłącznie w makrze. Warunek może zostać utworzony w oparciu o:

- wartości systemowe
- wartości parametrów
- daty

Ogólna postać:

```
%if warunek1 %then %do;
	kod SAS;
	kod SAS;
%end;
%else %if warunek2 %then %do;
	kod SAS;
%end;
%else %do;
	kod SAS;
%end;
```


In [1]:
libname bib "/folders/myfolders/przetwarzanie-danych-SAS/";

SAS Connection established. Subprocess id is 3516



Prosty program - wywołaj kod jeśli dzisiaj jest poniedziałek:

In [6]:
%macro raport;
%if &sysday=Monday %then %do;
proc means data=bib.bank;
run;
%end;
%mend;

%raport;


Variable,Label,N,Mean,Std Dev,Minimum,Maximum
wiek praca stan_cywil saldo kontakt dzien miesiac czas licz_kontakt dni_kontakt licz_kontakt_poprz wynik_poprz,wiek praca stan_cywil saldo kontakt dzien miesiac czas licz_kontakt dni_kontakt licz_kontakt_poprz wynik_poprz,45211 44923 45211 45211 32191 45211 45211 45211 45211 45211 45211 8252,40.9362102 5.6851947 1.8322753 1362.27 1.0902737 15.8064188 6.1446551 258.1630798 2.7638407 40.1978280 0.5803234 2.0398691,10.6187620 2.9754947 0.6082298 3044.77 0.2865779 8.3224762 2.4080340 257.5278123 3.0980209 100.1287460 2.3034410 0.6360368,18.0000000 1.0000000 1.0000000 -8019.00 1.0000000 1.0000000 1.0000000 0 1.0000000 -1.0000000 0 1.0000000,95.0000000 10.0000000 3.0000000 102127.00 2.0000000 31.0000000 12.0000000 4918.00 63.0000000 871.0000000 275.0000000 3.0000000


Ten sam program dla innego dnia tygodnia nie zadziała:

In [7]:
%macro raport;
%if &sysday=Friday %then %do;
proc means data=bib.bank;
run;
%end;
%mend;

%raport;


Zastosujmy przetwarzanie warunkowe do ulepszenia makra imputującego wartości.

In [11]:
%macro imputacja();

proc means data=bib.bank_braki n nmiss median;
var saldo;
class stan_cywil;
output out=saldo_stat median=saldo_mediana;
run;

data bib.bank_imputacja;
set bib.bank_braki;
run;

data _null_;
 set saldo_stat nobs=nobs;
 call symputx("n", nobs);
run;

%do i=1 %to &n;

data _null_;
set saldo_stat(obs=&i);
call symputx("mz_saldo", saldo_mediana);
call symputx("mz_kat", stan_cywil);
run;

%put &mz_saldo;

data bib.bank_imputacja;
set bib.bank_imputacja;
if saldo=. and stan_cywil=&mz_kat then saldo=&mz_saldo;
run;

%end;

proc means data=bib.bank_imputacja n nmiss median;
var saldo;
class stan_cywil;
run;


%mend;

%imputacja();